<img width="1347" height="392" alt="image" src="https://github.com/user-attachments/assets/df0f71b5-7bae-4751-a85c-92f9cfbf61af" />
<a href="https://github.com/WilmaDarc/volta-zero-/tree/main">

**Profissão: Cientista de Dados**

**Módulo 27 | PCA |  Exercício 1**


**Aluna:[Wilma Darc Alves de Farias](www.linkedin.com/in/wilma-farias-66a15962)<br>** 
**Data: 23 de dezembro de 2025.**

# PCA - Tarefa 01: *HAR* com PCA

Vamos trabalhar com a base da demonstração feita em aula, mas vamos explorar um pouco melhor como é o desempenho da árvore variando o número de componentes principais.

In [9]:
import pandas as pd
from pathlib import Path
from collections import Counter
from sklearn.tree import DecisionTreeClassifier
from sklearn.decomposition   import PCA
from sklearn.metrics         import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

In [10]:
base = Path(r"C:\EBAC CIENCIA DE DADOS\human+activity+recognition+using+smartphones\UCI HAR Dataset")

filename_features = base / "features.txt"
filename_labels = base / "activity_labels.txt"
filename_subtrain = base / "train" / "subject_train.txt"
filename_xtrain = base / "train" / "X_train.txt"
filename_ytrain = base / "train" / "y_train.txt"
filename_subtest = base / "test" / "subject_test.txt"
filename_xtest = base / "test" / "X_test.txt"
filename_ytest = base / "test" / "y_test.txt"

# ler features e criar lista de nomes
df_features = pd.read_csv(filename_features, header=None, sep=r'\s+')
features = df_features.iloc[:, 1].astype(str).tolist()

# garantir nomes únicos para features duplicadas
counts = Counter()
unique_features = []
for name in features:
    counts[name] += 1
    unique_features.append(name if counts[name] == 1 else f"{name}_{counts[name]}")

# ler labels de atividade
labels = pd.read_csv(filename_labels, header=None, sep=r'\s+', names=['cod_label', 'label'])

# ler subjects e targets como Series
subject_train = pd.read_csv(filename_subtrain, header=None, names=['subject_id']).iloc[:, 0]
y_train = pd.read_csv(filename_ytrain, header=None, names=['cod_label']).iloc[:, 0]
subject_test = pd.read_csv(filename_subtest, header=None, names=['subject_id']).iloc[:, 0]
y_test = pd.read_csv(filename_ytest, header=None, names=['cod_label']).iloc[:, 0]

# ler X_train e X_test e ajustar colunas conforme necessário
X_train = pd.read_csv(filename_xtrain, header=None, sep=r'\s+')
if X_train.shape[1] == len(unique_features):
    X_train.columns = unique_features
else:
    features_adj = unique_features[:X_train.shape[1]] + [f"var_{i}" for i in range(X_train.shape[1] - len(unique_features))]
    X_train.columns = features_adj

X_test = pd.read_csv(filename_xtest, header=None, sep=r'\s+')
if X_test.shape[1] == len(unique_features):
    X_test.columns = unique_features
else:
    X_test.columns = unique_features[:X_test.shape[1]]

## Árvore de decisão

Rode uma árvore de decisão com todas as variáveis, utilizando o ```ccp_alpha=0.001```. Avalie a acurácia nas bases de treinamento e teste. Avalie o tempo de processamento.

In [11]:
%%time
# Medindo o tempo de processamento
# A partir deste ponto, será cronometrado o tempo necessário para executar o código abaixo.

# Criação do classificador de árvore de decisão com ccp_alpha=0.001
clf = DecisionTreeClassifier(ccp_alpha=0.001)

# Treinamento do classificador utilizando os dados de treinamento
clf.fit(X_train, y_train)

# Avaliação da acurácia do classificador nos dados de treinamento
print(f'Acurácia na base de treinamento: {clf.score(X_train, y_train)}')
# Avaliação da acurácia do classificador nos dados de teste
print(f'Acurácia na base de teste: {clf.score(X_test, y_test)}\n')


Acurácia na base de treinamento: 0.9757889009793254
Acurácia na base de teste: 0.8795385137427892

CPU times: total: 6.44 s
Wall time: 6.52 s


## Árvore com PCA

Faça uma análise de componemtes principais das variáveis originais. Utilize apenas uma componente. Faça uma árvore de decisão com esta componente como variável explicativa.

- Avalie a acurácia nas bases de treinamento e teste
- Avalie o tempo de processamento

In [12]:
%%time
# Aplica o PCA com 1 componente aos dados de treinamento
prcomp = PCA(n_components=1).fit(X_train)

# Transforma os dados de treinamento e teste utilizando as componentes principais encontradas pelo PCA
pc_treino = prcomp.transform(X_train)
pc_teste  = prcomp.transform(X_test)

# Imprime a forma dos dados de treinamento e teste após a transformação
print(f'Dimensões da base de treinamento: {pc_treino.shape}')
print(f'Dimensões da base de teste: {pc_teste.shape}')

# Inicializa um classificador de árvore de decisão com ccp_alpha=0.001 e treina-o com os dados de treinamento transformados
clf = DecisionTreeClassifier(ccp_alpha=0.001)
clf.fit(pc_treino, y_train)

# Calcula e imprime a acurácia do classificador nos dados de treinamento e teste
print(f'Acurácia na base de treinamento: {clf.score(pc_treino, y_train)}')
print(f'Acurácia na base de teste: {clf.score(pc_teste, y_test)}\n')

Dimensões da base de treinamento: (7352, 1)
Dimensões da base de teste: (2947, 1)
Acurácia na base de treinamento: 0.499727965179543
Acurácia na base de teste: 0.45707499151679676

CPU times: total: 641 ms
Wall time: 514 ms


## Testando o número de componentes

Com base no código acima, teste a árvore de classificação com pelo menos as seguintes possibilidades de quantidades de componentes: ```[1, 2, 5, 10, 50]```. Avalie para cada uma delas:

- Acurácia nas bases de treino e teste
- Tempo de processamento


In [13]:
%%time

componentes = [1, 2, 5, 10, 50]

# Loop sobre os diferentes números de componentes
for n in componentes:
    # Executa o PCA com o número de componentes atual
    prcomp = PCA(n_components=n).fit(X_train)

    # Transforma os dados de treinamento e teste nos componentes principais
    pc_treino = prcomp.transform(X_train)
    pc_teste  = prcomp.transform(X_test)

    # Imprime as dimensões dos dados transformados
    print(f'Dimensões da base de treinamento: {pc_treino.shape}')
    print(f'Dimensões da base de teste: {pc_teste.shape}')

    # Cria e treina um classificador de árvore de decisão
    clf = DecisionTreeClassifier(ccp_alpha=0.001)
    clf.fit(pc_treino, y_train)

    # Avalia a acurácia na base de treinamento e teste
    print(f'Acurácia na base de treinamento: {clf.score(pc_treino, y_train)}')
    print(f'Acurácia na base de teste: {clf.score(pc_teste, y_test)}\n')

Dimensões da base de treinamento: (7352, 1)
Dimensões da base de teste: (2947, 1)
Acurácia na base de treinamento: 0.499727965179543
Acurácia na base de teste: 0.45707499151679676

Dimensões da base de treinamento: (7352, 2)
Dimensões da base de teste: (2947, 2)
Acurácia na base de treinamento: 0.6127584330794341
Acurácia na base de teste: 0.5846623685103495

Dimensões da base de treinamento: (7352, 5)
Dimensões da base de teste: (2947, 5)
Acurácia na base de treinamento: 0.8460282916213275
Acurácia na base de teste: 0.7889379029521547

Dimensões da base de treinamento: (7352, 10)
Dimensões da base de teste: (2947, 10)
Acurácia na base de treinamento: 0.8926822633297062
Acurácia na base de teste: 0.8238887003732609

Dimensões da base de treinamento: (7352, 50)
Dimensões da base de teste: (2947, 50)
Acurácia na base de treinamento: 0.919341675734494
Acurácia na base de teste: 0.822870715982355

CPU times: total: 4.08 s
Wall time: 2.18 s


## Conclua

- O que aconteceu com a acurácia?
- O que aconteceu com o tempo de processamento?

>De início, ao reduzir a dimensionalidade utilizando a técnica de Análise de Componentes Principais (PCA), observamos que a principal mudança e mais perceptível ocorre em relação ao tempo de processamento. A árvore de decisão executada com o conjunto de dados completo, contendo todas as 561 variáveis, levou aproximadamente 6,52 segundos para ser executada, resultando em uma acurácia de 88% na base de teste. Entretanto, ao utilizar o PCA com apenas um componente, o tempo de processamento foi reduzido para pouco menos de 514 ms, porém a acurácia na base de testes foi de 45%.
>
>Aumentando o número de componentes, foi possível aumentar gradualmente a acurácia na base de testes. Com 50 componentes, foi atingida uma acurácia de até 82%, com um tempo de processamento de aproximadamente 2.18 segundos para executar todas as 5 combinações de números diferentes de componentes. Portanto, podemos concluir que é possível obter uma acurácia semelhante à árvore com todas as variáveis do conjunto de dados, porém com um tempo de processamento reduzido em menos da metade. É importante ressaltar que os componentes são formados a partir da combinação linear das variáveis originais.